In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("hf://datasets/Kaludi/Customer-Support-Responses/Customer-Support.csv")

/Users/aadityaraj/Library/Python/3.12/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Data Preparation 

In [3]:
df

,query,response
0,My order hasn't arrived yet.,We apologize for the inconvenience. Can you pl...
1,I received a damaged product.,We apologize for the inconvenience. Can you pl...
2,I need to return an item.,Certainly. Please provide your order number an...
3,I want to change my shipping address.,No problem. Can you please provide your order ...
4,I have a question about my bill.,We'd be happy to help. Can you please provide ...
...,...,...
69,How do I schedule a consultation or appointment?,We'd be happy to help. Can you please provide ...
70,Can I get a copy of my receipt?,Certainly. Can you please provide your order n...
71,Can I use a competitor's coupon at your store?,"In some cases, we may accept competitor coupon..."
72,Do you have a recycling program?,"Yes, we do have a recycling program. Can you p..."


In [5]:
context = [data for data in df["response"]]
context

['We apologize for the inconvenience. Can you please provide your order number so we can investigate?',
 'We apologize for the inconvenience. Can you please provide a photo of the damaged product so we can assist you further?',
 'Certainly. Please provide your order number and reason for return, and we will provide you with instructions on how to proceed.',
 "No problem. Can you please provide your order number and the new shipping address you'd like to use?",
 "We'd be happy to help. Can you please provide your account number and a brief description of your question?",
 "We're sorry to hear that. Can you please provide your account email and the name of the subscription you'd like to cancel?",
 'We understand your concern. Please provide your order number and the reason for your refund request so we can assist you further.',
 'Of course. Can you please provide your order number so we can check the current status for you?',
 'We apologize for the inconvenience. Can you please provide y

In [13]:
# 'qas': [{'answers': [{'answer_start': 115, 'text': 'Balkan peninsula'}],
#      'id': '572611d489a1e219009ac1e0',
#      'is_impossible': False,
#      'question': 'On what peninsula is Greece located?'},
#     {'answers': [{'answer_start': 287, 'text': 'nine'}],
#      'id': '572611d489a1e219009ac1e1',
#      'is_impossible': False,
#      'question': 'How many geographic regions make up Greece?'},
#     {'answers': [{'answer_start': 708, 'text': '8,498 mi'}],
#      'id': '572611d489a1e219009ac1e2',
#      'is_impossible': False,
#      'question': 'How long is the coastline of Greece?'},
#     {'answers': [{'answer_start': 774, 'text': '227'}],
#      'id': '572611d489a1e219009ac1e3',
#      'is_impossible': False,
#      'question': "How many of Greece's islands are inhabited?"},
#     {'answers': [{'answer_start': 839, 'text': 'Mount Olympus'}],
#      'id': '572611d489a1e219009ac1e4',
#      'is_impossible': False,
#      'question': 'What is the tallest mountain in Greece?'}]},

In [14]:
formated_data = []
cnt=0
for que, ans in zip(df["query"],df["response"]):
    cnt+=1
    formated_data.append(
            {
            "context": ans,
            "qas":[
                    {
                        "answers":[{"answer_start": 0, "text": ans}],
                        "id": cnt,
                        "is_impossible":False,
                        "question": que
                    },
                 ]
            }
    )
    

formated_data

[{'context': 'We apologize for the inconvenience. Can you please provide your order number so we can investigate?',
  'qas': [{'answers': [{'answer_start': 0,
      'text': 'We apologize for the inconvenience. Can you please provide your order number so we can investigate?'}],
    'id': 1,
    'is_impossible': False,
    'question': "My order hasn't arrived yet."}]},
 {'context': 'We apologize for the inconvenience. Can you please provide a photo of the damaged product so we can assist you further?',
  'qas': [{'answers': [{'answer_start': 0,
      'text': 'We apologize for the inconvenience. Can you please provide a photo of the damaged product so we can assist you further?'}],
    'id': 2,
    'is_impossible': False,
    'question': 'I received a damaged product.'}]},
 {'context': 'Certainly. Please provide your order number and reason for return, and we will provide you with instructions on how to proceed.',
  'qas': [{'answers': [{'answer_start': 0,
      'text': 'Certainly. Please

In [112]:
train_data = formated_data[0:len(formated_data)-10]
test_data = formated_data[len(formated_data)-10:]

In [113]:
print(len(train_data))
print(len(test_data))

64
10


# Transformers 

In [26]:
# pip install simpletransformers

In [114]:
import logging
from simpletransformers.question_answering import QuestionAnsweringModel, QuestionAnsweringArgs

In [115]:
#train_args are the parameters the QuestionAnswerringModel will use 
train_args = {
    'overwrite_output_dir': True,
    "evaluate_during_training": True,
    "max_seq_length": 128,
    "num_train_epochs": 25, #25, after experimentations
    "evaluate_during_training_steps": 500,
    "save_model_every_epoch": False,
    "save_eval_checkpoints": False,
    "n_best_size":4, #batch_size is another important argument
    "train_batch_size": 16,
    "eval_batch_size": 16
}

In [116]:
model = QuestionAnsweringModel("bert",
                               "bert-base-cased", 
                               args = train_args,
                               use_cuda=False) # I will use GPU for faster performance

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [117]:
model.train_model(train_data, eval_data=test_data)

Epochs 1/25. Running Loss:    4.2518: 100%|███████| 4/4 [00:06<00:00,  1.72s/it]

convert squad examples to features: 100%|█████| 10/10 [00:00<00:00, 2455.25it/s]

add example index and unique id: 100%|██████| 10/10 [00:00<00:00, 293307.97it/s]

Epochs 2/25. Running Loss:    2.2175: 100%|███████| 4/4 [00:06<00:00,  1.73s/it]

convert squad examples to features: 100%|█████| 10/10 [00:00<00:00, 2012.14it/s]

add example index and unique id: 100%|██████| 10/10 [00:00<00:00, 346636.69it/s]

Epochs 3/25. Running Loss:    0.8086: 100%|███████| 4/4 [00:06<00:00,  1.70s/it]

convert squad examples to features: 100%|█████| 10/10 [00:00<00:00, 2001.19it/s]

add example index and unique id: 100%|██████| 10/10 [00:00<00:00, 384798.53it/s]

Epochs 4/25. Running Loss:    0.1787: 100%|███████| 4/4 [00:06<00:00,  1.71s/it]

convert squad examples to features: 100%|██████| 10/10 [00:00<00:00, 854.62it/s]

add example index and unique id: 100%|██████| 10/10 [00:00<00:00, 343795.41it/s]

Epochs 5/25. Run

(100,
 {'global_step': [4,
   8,
   12,
   16,
   20,
   24,
   28,
   32,
   36,
   40,
   44,
   48,
   52,
   56,
   60,
   64,
   68,
   72,
   76,
   80,
   84,
   88,
   92,
   96,
   100],
  'correct': [9,
   9,
   9,
   9,
   10,
   10,
   10,
   10,
   10,
   10,
   10,
   10,
   10,
   10,
   10,
   10,
   10,
   10,
   10,
   10,
   10,
   10,
   10,
   10,
   10],
  'similar': [1,
   1,
   1,
   1,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0],
  'incorrect': [0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0],
  'train_loss': [4.251778602600098,
   2.2174644470214844,
   0.8086005449295044,
   0.1786598414182663,
   0.060154080390930176,
   0.009751392528414726,
   0.002881144406273961,
   0.0010526429396122694,
   0.0006741434335708618,
   0.00046707422006875277,
   0.00033541314769536257,
   0.0002

In [118]:
# Evaluate the model
result, texts = model.eval_model(test_data)

Running Evaluation: 100%|█████████████████████████| 1/1 [00:00<00:00,  3.19it/s]


In [119]:
print(result)
# print(texts)

{'correct': 10, 'similar': 0, 'incorrect': 0, 'eval_loss': -5.143555641174316}


#  INFERENCE

In [120]:
# Load model from training checkpoint
from simpletransformers.question_answering import QuestionAnsweringModel, QuestionAnsweringArgs
 
model = QuestionAnsweringModel("bert", "outputs/best_model", use_cuda=False)

In [123]:
# Make predictions with the model
to_predict = [
    {
        "context": " ".join(k,v for k,v in data),
        "qas": [
            {
                "question": "Is my item in stock?",
                "id": "0",
            }
        ],
    }
]

answers, probabilities = model.predict(to_predict, n_best_size=2)
print(answers)
# if len(answers)>0:
#     print(answers[0]["answer"][0])
#     print(answers[0]["answer"][0])

Running Prediction: 100%|█████████████████████████| 1/1 [00:00<00:00, 12.52it/s]

[{'id': '0', 'answer': ['Inventory has so much stocks of all the products, we need to get the stock id or item id to get the info for any stocks.', 'we need to get the stock id or item id to get the info for any stocks.']}]
